In [1]:
import os
import glob
import tensorflow as tf

In [2]:
os.makedirs('person_proto', exist_ok=True)

In [3]:
%%writefile ./person_proto/person.proto
syntax = "proto3";
message Person {
  string name = 1;
  int32 id = 2;
  repeated string email = 3;
}

Overwriting ./person_proto/person.proto


In [4]:
!protoc ./person_proto/person.proto --python_out=. --descriptor_set_out=./person_proto/person.desc --include_imports

In [5]:
glob.glob('./person_proto/person*')

['./person_proto\\person.desc',
 './person_proto\\person.proto',
 './person_proto\\person_pb2.py']

In [6]:
from person_proto.person_pb2 import Person

person = Person(name="AAA", id=123, email=["a@b.com"])  # create a Person
person

name: "AAA"
id: 123
email: "a@b.com"

In [7]:
person.name

'AAA'

In [8]:
person.name = "BBB"
person

name: "BBB"
id: 123
email: "a@b.com"

In [9]:
s = person.SerializeToString()
s

b'\n\x03BBB\x10{\x1a\x07a@b.com'

In [10]:
person2 = Person()
person2.ParseFromString(s)
person2

name: "BBB"
id: 123
email: "a@b.com"

In [11]:
person2 == person

True

## Use custom protobuf in TensorFlow

In [12]:
person_tf = tf.io.decode_proto(
    bytes=s,
    message_type='Person',
    field_names=['name', 'id', 'email'],
    output_types=[tf.string, tf.int32, tf.string],
    descriptor_source="./person_proto/person.desc"
)

person_tf.values

[<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'BBB'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([123])>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'a@b.com'], dtype=object)>]